In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent.parent
sys.path.append(str(BASE_DIR))

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3T
import numpy as np
import pandas as pd

In [5]:
DIR = BASE_DIR/'scripts_res'/'Case3TAC3'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res/Case3TAC3')

In [6]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

def func(x1,x2,x3,x4,x5):
    x = np.array([x1,x2,x3,x4,x5])
    obj, c1, c2, c3, c4= Case3T(x)
    cons = [c1,c2,c3,c4]
    return -obj,cons

In [7]:
obj_list = []
x_1_list = []
x_2_list = []
x_3_list = []
x_4_list = []
x_5_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []
con_4_list  = []
all_res_list = []

strategy = 2
nc_lb = np.array([-1,-1,-2,0,])
nc_ub = np.array([2,  4, 3,1.5])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 12),'x2': (-20, 22),'x3': (-10, 10),'x4': (-12, 20),'x5': (-15, 18),}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

for i in range(10):
    optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=2,random_state=i)
    try:
        optimizer.maximize(init_points=100, n_iter=200,p_hyper=[[4, 2.5, 0.78, 0.95,1e-5]],
                   acquisition_function=acquisition_function,dir=DIR/f'Case3T_{i}_Strategy{strategy}.csv', strategy=strategy)#
        print(optimizer.max)
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        x_3_list.extend([optimizer.max['params']['x3']])
        x_4_list.extend([optimizer.max['params']['x4']])
        x_5_list.extend([optimizer.max['params']['x5']])  
        obj_list.extend([optimizer.max['target']])                                  
        con_1_list.extend([optimizer.max['constraint'][0]])
        con_2_list.extend([optimizer.max['constraint'][1]])
        con_3_list.extend([optimizer.max['constraint'][2]])
        con_4_list.extend([optimizer.max['constraint'][3]])
        all_res = [item['target'] for item in optimizer.res]
        all_fea = [item['allowed'] for item in optimizer.res]
        modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
        all_res_list.append(modified_res)    
    except Exception as e:
        print(e)
        obj_list.append(None)
        x_1_list.append(None)
        x_2_list.append(None)
        x_3_list.append(None)
        x_4_list.append(None)
        x_5_list.append(None)                        
        con_1_list.append(None)
        con_2_list.append(None)
        con_3_list.append(None)
        con_4_list.append(None) 
        modified_res = [-np.inf for _ in range(100+200)]
        all_res_list.append(modified_res)   
df2 = pd.DataFrame(all_res_list, columns=[list(range(100+200))])               
df = pd.DataFrame({
'obj': obj_list,
'x_1': x_1_list,
'x_2': x_2_list,
'x_3': x_3_list,
'x_4': x_4_list,
'x_5': x_5_list,
'con_1': con_1_list,
'con_2': con_2_list,
'con_3': con_3_list,
'con_4': con_4_list
})
df.to_csv(DIR/f'Case3T_{i}_Strategy{strategy}_Results.csv', index=False)
df2.to_csv(DIR/f'Case3T_{i}_Strategy{strategy}_Results2.csv', index=False)

|   iter    |  target   |  allowed  |    x1     |    x2     |    x3     |    x4     |    x5     |
-------------------------------------------------------------------------------------------------
| 1         | -21.36    | False     | -0.495    | -12.23    | -1.5      | 5.44      | 14.54     |
| 2         | -21.2     | False     | 9.705     | -7.61     | 9.3       | 15.04     | 3.645     |
| 3         | -16.65    | False     | -4.575    | 7.09      | 1.7       | -1.28     | -2.625    |
| 4         | -20.35    | False     | 4.435     | 18.85     | -7.1      | 5.12      | 0.345     |
| 5         | -21.2     | False     | 7.835     | 10.45     | 4.5       | -11.2     | -14.18    |
| 6         | -18.22    | False     | 10.89     | 16.75     | -9.1      | 3.84      | 14.87     |
| 7         | -21.47    | False     | -2.195    | -17.69    | -0.5      | 3.52      | -10.54    |
| 8         | -21.46    | False     | 7.495     | -13.49    | 2.9       | 14.4      | -9.885    |
| 9         | -20.21

In [8]:
df = pd.read_csv(DIR/'Case3T_9_Strategy2_Results.csv')
df.describe()

,obj,x_1,x_2,x_3,x_4,x_5,con_1,con_2,con_3,con_4
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,-5.264898,1.142999,-1.121260,0.482370,1.368083,0.303549,0.375867,1.176950,1.760902,0.761505
std,4.205772,1.909811,2.649230,2.586146,2.528590,3.522026,0.701785,0.714175,1.239825,0.394334
min,-14.690165,-2.768257,-5.983739,-4.042332,-1.068005,-5.970196,-0.653569,0.163722,-0.141383,0.000377
25%,-5.147885,0.970789,-1.230480,0.383928,0.061116,-0.752641,0.021809,0.763472,0.610628,0.557558
50%,-4.275481,1.039354,-1.106695,1.087243,0.713563,0.219647,0.336054,1.093919,2.449057,0.784841
75%,-2.925759,1.763413,-1.027077,1.859078,1.745171,0.328202,0.471897,1.477286,2.773259,1.010426
max,-0.009073,4.990281,4.191161,4.002690,8.047864,7.886826,1.813943,2.384767,2.925684,1.355236
